# Prepare Path Retention Database
This notebook tries to read taz centriod files and regenerate all a list of OD pairs to calculate the path retention using the RoadwaySim. A big trip input table is prepared just by enumerating all OD trip combinations. Then the roadwaySim is run but the results are stored in a densed table using json data structure or a small database or csv files instead of a explicit path retention with time.

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import geopandas as gpd

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/%autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2
pd.options.display.max_columns = None  # display all columns

In [2]:
# need to set this environmental path everytime you moves the project root folder
os.environ['project_root'] = '/Users/geekduck/Dropbox/gt_survey'

sys.path.append(
    os.environ['project_root']
)

In [3]:
from carpoolsim.basic_settings import *

In [4]:
# # rt = '/Users/diyi93/Dropbox/My Mac (Diyi’s MacBook Pro)'
# rt = '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)'

# # need to set this environmental path everytime you moves the project root folder
# os.environ['PROJ_LIB'] = os.path.join(rt, 'Desktop/gra/CarpoolSim')
# # another root directory for your data
# os.environ['PORJ_DATA'] = os.path.join(rt, 'Desktop/Data/GIS/Atlanta/ARC ABM-20')

# # trip data directory
# os.environ['abm_data'] = os.path.join(rt, 'Desktop/Data/ABM/ABM processed data/')
# # network data directory
# os.environ['network_data'] = os.path.join(rt, 'Desktop/Data/GIS/Atlanta/2020')
# # project root
# os.environ['project_root'] = os.path.join(rt, 'Desktop/gra/CarpoolSim')

1. Import centriods and generate all od pairs

This is $5873 \times {622} = 3653006$, less than four million trips to query.

In [5]:
# load taz centriod nodes, pnr nodes and  nodes
taz_centriod_nodes = gpd.read_file(os.environ["taz"])
pnr_nodes = gpd.read_file(os.environ["parking_lots"])

# load original links and nodes (containing all connector and centriods, which construct a whole network)
df_nodes_raw = gpd.read_file(os.environ['network_nodes'])
df_links_raw = gpd.read_file(os.environ['network_links'])

In [8]:
display(taz_centriod_nodes.head())
display(df_links_raw.head())

,OBJECTID_1,OBJECTID,MTAZ10,CTAZ10,TRACT10,TAD10,SD10,FIPS,COUNTY,ACRES,GlobalID,last_edite,Shape_Leng,Shape_Area,TAZS,TAZS_int,layer,path,geometry
0,1944,1944,1944,13067287,1.306703e+10,13067015,31,13067,Cobb,203.230808,{C13756E7-1C10-4B81-87F3-B571E3375C44},None,0.040636,0.000080,1944,1944,None,None,"POLYGON ((-84.67700 34.07407, -84.67706 34.073..."
1,1945,1945,1945,13067287,1.306703e+10,13067015,31,13067,Cobb,118.580318,{8D479E02-EDB7-4719-A536-8146A6B81B6D},None,0.027760,0.000047,1945,1945,None,None,"POLYGON ((-84.67876 34.07370, -84.67820 34.073..."
2,1946,1946,1946,13067286,1.306703e+10,13067015,31,13067,Cobb,587.259136,{E7E20B44-0D06-438F-8748-2746464CC5C5},None,0.068585,0.000232,1946,1946,None,None,"POLYGON ((-84.67642 34.07830, -84.67494 34.078..."
3,1947,1947,1947,13067286,1.306703e+10,13067015,31,13067,Cobb,71.448426,{A2A315F1-A5CD-4B8B-B9F8-06F33FC2E9CF},None,0.022855,0.000028,1947,1947,None,None,"POLYGON ((-84.66004 34.06479, -84.66006 34.064..."
4,1948,1948,1948,13067285,1.306703e+10,13067015,31,13067,Cobb,290.041893,{CCE4641A-542D-450A-9A8C-688DA3353D1C},None,0.043399,0.000115,1948,1948,None,None,"POLYGON ((-84.65346 34.06250, -84.65389 34.062..."


,N,X,Y,STATION,STAFLAG,PNR,PNR_MAJOR,GEOMETRYSO,lat,lon,geometry
0,1,2.220979e+06,1.483122e+06,None,0,0,0,1,34.076861,-84.416355,POINT (-84.41636 34.07686)
1,2,2.222861e+06,1.479567e+06,None,0,0,0,1,34.067103,-84.410110,POINT (-84.41011 34.06710)
2,3,2.226472e+06,1.484525e+06,None,0,0,0,1,34.080752,-84.398224,POINT (-84.39822 34.08075)
3,4,2.233517e+06,1.483418e+06,None,0,0,0,1,34.077750,-84.374950,POINT (-84.37495 34.07775)
4,5,2.236754e+06,1.485119e+06,None,0,0,0,1,34.082443,-84.364269,POINT (-84.36427 34.08244)


In [5]:
source_lst = list(range(5873))  # a list of sources (here is all taz number)
destinations_lst = list(range(54,70))  # a list of interested taz destinations

## Generate the Network Graph

After we got the task. We need to run the a dijkstra algorithm once for each source. (Notice that if the number of destinations are small then we can compute the reversed graph to save computation time)


Let's try to solve shortest paths for the above tasks. **Frist of all, we need to get the network graph**

In [6]:
import os
import sys

code_root = os.path.join(os.getcwd(), "core_script")
sys.path.append(code_root)

In [7]:
# import BikeSim module
from core_script.getPaths_bikewaySim import (
    initialize_abm15_links,
    build_carpool_network,
)

In [8]:
print('** Initialize carpool network **')
df_links = initialize_abm15_links(drop_connector=False)
df_links.head(2)

** Initialize carpool network **


,A,B,A_B,geometry,SPEED_LIMI,DISTANCE,NAME,FACTYPE,Ax,Ay,A_lat,A_lon,Bx,By,B_lat,B_lon,minx_sq,miny_sq,maxx_sq,maxy_sq
0,1,65666,1_65666,"LINESTRING (2220978.672 1483122.486, 2221582.3...",35,0.23355,None,0,2.220979e+06,1.483122e+06,34.076861,-84.416355,2.221582e+06,1.484198e+06,34.07982,-84.41437,89.0,59.0,89.0,59.0
1,1,80483,1_80483,"LINESTRING (2220978.672 1483122.486, 2220402.3...",35,0.44684,None,0,2.220979e+06,1.483122e+06,34.076861,-84.416355,2.220402e+06,1.480835e+06,34.07057,-84.41824,89.0,59.0,89.0,59.0


In [9]:
# build networkx graph takes less than two minutes
print('** build RoadwaySim network **')
dict_bike = {'DG': build_carpool_network(df_links), 'links': df_links} # use default speed 45 mph

** build RoadwaySim network **


In [10]:
print(dict_bike['DG']['1']['65666'])
print(dict_bike['DG']['65666']['1'])

{'dist': 0.23355, 'name': None, 'forward': 0.40037142857142854, 'backward': 0.40037142857142854}
{'dist': 0.23355, 'name': None, 'backward': 0.40037142857142854, 'forward': 0.40037142857142854}


In [11]:
from core_script.getPaths_bikewaySim import run_batch_ods

In [12]:
# only record results to centroids
destination_lst = list(range(1,5874))
destination_lst = [ str(i) for i in destination_lst]
import time
t1 = time.time()
dists_dict, paths_dict = run_batch_ods(dict_bike['DG'], '1', destination_lst, weight='backward')
delta_t = time.time() - t1
print('It takes {} seconds to run'.format(delta_t))

It takes 0.09407782554626465 seconds to run


In [13]:
print(dists_dict['5000'])  # '5000' ==> '1'

67.01371002164502


In [14]:
# a shortest path from taz centroid 1 to taz centroid 2000
dists_dict, paths_dict = run_batch_ods(
    dict_bike['DG'], '1', destination_lst, weight='forward')
print(dists_dict['5000'])  # '1' ==> '5000'

67.55646746753247


# Summary
It seems like calculate a single taz source to all destinations is "fast" (~0.1 second). We can easily regenerate all shortest paths between any two centroids by enumerating every source once.

Also, multiprocessing is used to speed up the process...

In [94]:
import multiprocess  # not multiprocessing
import time
import pickle

In [95]:
destination_lst = list(range(1,5874))
destination_lst = [str(i) for i in destination_lst]

In [96]:
def get_shortest_paths(source_taz_lst):
    path_retention_dists, path_retention_paths = {}, {}
    t1 = time.time()
    taz_lst = []
    for taz in source_taz_lst:
        dists_dict, paths_dict = run_batch_ods(
            dict_bike['DG'], str(taz), destination_lst, weight='forward')
        path_retention_dists[str(taz)] = dists_dict
        path_retention_paths[str(taz)] = paths_dict
        taz_lst.append(str(taz))
    
    print(f'Finished searching {len(source_taz_lst)} tazs')
    # store values in disk instead of holding all of the memories
    db = {}
    db['dist'] = path_retention_dists
    db['path'] = path_retention_paths

    first_taz = source_taz_lst[0]
    last_taz_id = source_taz_lst[-1]
    folder = "build_graph/path_retention"
    file_name = f'paths_retention_{first_taz:04d}_{last_taz_id:04d}.pickle'
    fname = os.path.join(folder, file_name)
    with open(fname, 'wb') as dbfile:
        pickle.dump(db, dbfile)

In [97]:
# path_retention_dists, path_retention_paths = {}, {}
taz_lst = []
for i in range(58):
    _lst = list(range(i*100+1, (i+1)*100+1))
    taz_lst.append(_lst)
taz_lst.append(list(range(5801, 5874)))

In [98]:
count_tazs = 0
for taz in taz_lst:
    count_tazs += len(taz)
    # print(taz)
print(count_tazs)

5873


In [99]:
t0 = time.perf_counter()
with multiprocess.Pool(12) as pool:
    results = pool.map(get_shortest_paths, taz_lst)

d1 = time.perf_counter() - t0
print(f'It takes {d1/60:.1f} minutes to finish the run')

Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 100 tazs
Finished searching 1

# Store results in a database

At least two datasets are provided:
- Option 1: sqlite
- Option 2: PostgreSQL

In [100]:
import pickle
import os
import glob
import time

rt = '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)'

path_retention_name_pattern = os.path.join(
    rt, 
    'Desktop/gra/CarpoolSim/build_graph/path_retention',
    'paths_retention_*.pickle'
)

pickle_lst = glob.glob(path_retention_name_pattern)
pickle_lst = sorted(pickle_lst)
display(pickle_lst)

['/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0001_0100.pickle',
 '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0101_0200.pickle',
 '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0201_0300.pickle',
 '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0301_0400.pickle',
 '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0401_0500.pickle',
 '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0501_0600.pickle',
 '/Users/geekduck/Dropbox/My Mac (Diyi’s MacBook Pro)/Desktop/gra/CarpoolSim/build_graph/path_retention/paths_retention_0601_0700.pickle',
 '/Users/geekduck/Dropbox/M

In [101]:
len(pickle_lst)

59

In [102]:
from sqlalchemy import create_engine
import pickle

import numpy as np
import pandas as pd

In [103]:
from core_script.database.prepare_database import (
    convert_dict_to_row,
    batch_store_df
)

In [104]:
# engine = create_engine(db_url, executemany_mode="batch")
db_url = 'sqlite:///build_graph/path_retention.db'
engine = create_engine(db_url)

with engine.connect() as connection:
    print('successfully connected')

successfully connected


In [105]:
# load pickle files to the database
t0 = time.perf_counter()

lst = [(item, db_url) for item in pickle_lst]
# with multiprocess.Pool(16) as pool:
#     results = pool.starmap(batch_store_from_name, lst)

# just use loop to avoid racing conditions of multiprocessing...
for pk_name in pickle_lst:
    batch_store_from_name(pk_name, db_url)


t1 = time.perf_counter()
print(f'Loading jobs finished. It takes {(t1-t0)/60:.1f} minutes in total!')

start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (587300, 2)
Appended a whole batch data to the server!
start feeding data to database! Dataframe shape:  (5873

# Try to access the database

In [106]:
# test the database works as expected
engine = create_engine(db_url)

# drop table dists;
# engine.execute('drop table dists;')

In [120]:
from core_script.database.query_database import (
    query_od_info,
    execute_sql_command
)

In [121]:
results = query_od_info(engine, '1404', '2820')
print(results)

('1404', '2820', 10.90411714285714, ['1404', '25495', '1403', '10415', '1402', '65883', '14841', '10304', '81987', '10457', '10459', '14869', '10458', '10467', '10464', '20940', '10469', '10472', '10473', '10479', '83091', '83093', '10474', '13593', '11959', '11960', '83099', '83101', '11961', '14599', '81454', '81455', '14597', '14596', '81451', '36631', '2820'])


In [122]:
5873 * 5873

34492129

In [124]:
# should be 5873 * 5873 = 
sql_command = "SELECT count(*) FROM dists;"

tot_lines = execute_sql_command(engine, sql_command, mode="scalar")
print(tot_lines)

34492129


In [128]:
sql_command = "SELECT * FROM dists LIMIT 5;"

tot_lines = execute_sql_command(engine, sql_command, mode="fetchall")
display(tot_lines)

[('1', '1', 0.0, "['1']"),
 ('1', '2', 1.6642285714285716, "['1', '80483', '2']"),
 ('1', '3', 1.9914342857142857, "['1', '65666', '3']"),
 ('1', '4', 4.348765714285714, "['1', '65666', '3', '74491', '4']"),
 ('1', '5', 5.66832, "['1', '65666', '3', '74491', '4', '74487', '5']")]